# 值迭代算法
作者：stzhao
github: https://github.com/zhaoshitian

# 一、定义环境


In [6]:
import numpy as np
from envs.simple_grid import DrunkenWalkEnv

In [7]:
def all_seed(env,seed = 1):
    ## 这个函数主要是为了固定随机种子
    import numpy as np
    import random
    import os
    env.seed(seed) 
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed) 


In [8]:
env = DrunkenWalkEnv(map_name="theAlley")
all_seed(env, seed = 1)

# 二、价值迭代算法


In [9]:
def value_iteration(env, theta=0.005, discount_factor=0.9):
    Q = np.zeros((env.nS, env.nA)) # 初始化一个Q表格
    count = 0
    while True:
        delta = 0.0
        Q_tmp = np.zeros((env.nS, env.nA))
        for state in range(env.nS):
            for a in range(env.nA):
                accum = 0.0
                reward_total = 0.0
                for prob, next_state, reward, done in env.P[state][a]:
                    accum += prob* np.max(Q[next_state, :])
                    reward_total += prob * reward
                Q_tmp[state, a] = reward_total + discount_factor * accum
                delta = max(delta, abs(Q_tmp[state, a] - Q[state, a]))
        Q = Q_tmp*0.01+Q*0.99
        
        count += 1
        if delta < theta or count > 1000: # 这里设置了即使算法没有收敛，跑100次也退出循环
            break 
    return Q

In [10]:
Q = value_iteration(env)
print(Q)

[[4.87272219e-01 5.84770098e-01 1.26725524e+00 5.84770098e-01]
 [6.82267976e-01 1.62258234e+00 3.52488471e+00 1.62258234e+00]
 [1.97790943e+00 4.41868945e+00 9.12834729e+00 4.41868945e+00]
 [5.31249418e+00 1.12592607e+01 2.23598018e+01 1.12592607e+01]
 [1.23902169e+01 2.43497205e+01 5.27339179e+01 2.43497205e+01]
 [3.24457271e+01 6.50523578e+01 1.22993588e+02 6.50523578e+01]
 [7.63708403e+01 1.48274902e+02 2.78621356e+02 1.48274902e+02]
 [1.73556216e+02 3.35551113e+02 6.28994272e+02 3.35551113e+02]
 [3.91480912e+02 7.06832116e+02 1.41749407e+03 7.06832116e+02]
 [8.85497562e+02 1.70695376e+03 3.19317544e+03 1.70695376e+03]
 [1.99541350e+03 3.84196808e+03 7.18575462e+03 3.84196808e+03]
 [4.49076072e+03 8.64555403e+03 1.61691560e+04 8.64555403e+03]
 [2.82857774e+04 3.53602138e+04 3.63708475e+04 3.53602138e+04]]


In [11]:
policy = np.zeros([env.nS, env.nA]) # 初始化一个策略表格
for state in range(env.nS):
    best_action = np.argmax(Q[state, :]) #根据价值迭代算法得到的Q表格选择出策略
    policy[state, best_action] = 1

policy = [int(np.argwhere(policy[i]==1)) for i in range(env.nS) ]
print(policy)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


# 三、测试

In [12]:
num_episode = 1000 # 测试1000次
def test(env,policy):
    
    rewards = []  # 记录所有回合的奖励
    success = []  # 记录该回合是否成功走到终点
    for i_ep in range(num_episode):
        ep_reward = 0  # 记录每个episode的reward
        state = env.reset()  # 重置环境, 重新开一局（即开始新的一个回合） 这里state=0
        while True:
            action = policy[state]  # 根据算法选择一个动作
            next_state, reward, done, _ = env.step(action)  # 与环境进行一个交互
            state = next_state  # 更新状态
            ep_reward += reward
            if done:
                break
        if state==12: # 即走到终点
            success.append(1)
        else:
            success.append(0)
        rewards.append(ep_reward)
    acc_suc = np.array(success).sum()/num_episode
    print("测试的成功率是：", acc_suc)
    

In [13]:
test(env, policy)

测试的成功率是： 0.638
